# CS 335: Introduction to Large Language Models
## Assignment 01
### **Total Marks**: 100
### **Deadline**: Sunday, 3rd March, 2024, 11:59 PM
### **Name**: Muhammad Talha
### **ID**: mj06974

#Instructions

1. Please rename your notebook as *Assignment_1_aa1234.ipynb* before the final submission. Notebooks which do not follow appropriate naming convention will not be graded.

2. Please submit your own work. If you have any questions, please feel free to reach out to the course instructors or RA.



# Assignment Overview

In this assignment, you are required to fine tune a LLM model of your that classifies which human value category a textual arguement belongs to. Your model will evaluated against 1-baseline, random-baseline results on the following dataset: test, Nahjalbalagha, Zhihu


# Setup



In [14]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score

# Download Files


##Evaluator

In [15]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py

--2024-02-29 17:06:01--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 8708 (8.5K) [text/plain]
Saving to: ‘evaluator.py.2’

evaluator.py.2      100%[===================>]   8.50K  --.-KB/s    in 0s      

2024-02-29 17:06:02 (90.5 MB/s) - ‘evaluator.py.2’ saved [8708/8708]



## 1-Baseline

In [16]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py

--2024-02-29 17:06:02--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... 

connected.
HTTP request sent, awaiting response... 200 OK
Length: 3613 (3.5K) [text/plain]
Saving to: ‘1-baseline.py.2’

1-baseline.py.2     100%[===================>]   3.53K  --.-KB/s    in 0s      

2024-02-29 17:06:02 (83.3 MB/s) - ‘1-baseline.py.2’ saved [3613/3613]



## Random-Baseline

In [17]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py

--2024-02-29 17:06:02--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 5442 (5.3K) [text/plain]
Saving to: ‘random-baseline.py.2’

random-baseline.py. 100%[===================>]   5.31K  --.-KB/s    in 0s      

2024-02-29 17:06:03 (30.9 MB/s) - ‘random-baseline.py.2’ saved [5442/5442]



## Dataset Files

In [18]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://zenodo.org/api/records/10564870/files-archive

--2024-02-29 17:06:03--  https://zenodo.org/api/records/10564870/files-archive
Resolving zenodo.org (zenodo.org)... 

188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘files-archive.2’

files-archive.2         [  <=>               ]   7.25M   591KB/s               ^C


In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!unzip files-archive -d Dataset

Archive:  files-archive
 extracting: Dataset/value-categories.json  
 extracting: Dataset/arguments-validation-zhihu.tsv  
 extracting: Dataset/level1-labels-training.tsv  
 extracting: Dataset/arguments-test-nahjalbalagha.tsv  
 extracting: Dataset/arguments-test.tsv  
 extracting: Dataset/arguments-training.tsv  
 extracting: Dataset/arguments-validation.tsv  
 extracting: Dataset/labels-test-nahjalbalagha.tsv  
 extracting: Dataset/labels-test-nyt.tsv  
 extracting: Dataset/labels-test.tsv  
 extracting: Dataset/labels-training.tsv  
 extracting: Dataset/level1-labels-test-nahjalbalagha.tsv  
 extracting: Dataset/level1-labels-test.tsv  
 extracting: Dataset/level1-labels-test-nyt.tsv  
 extracting: Dataset/labels-validation.tsv  
 extracting: Dataset/labels-validation-zhihu.tsv  
 extracting: Dataset/meta-arguments-g.tsv  
 extracting: Dataset/meta-arguments-c.tsv  
 extracting: Dataset/meta-arguments-e.tsv  
 extracting: Dataset/meta-arguments-f.tsv  
 extracting: Dataset/level1-l


 extracting: Dataset/README.md       
 extracting: Dataset/comments-level1.tsv  


In [ ]:
!mkdir Dataset/zhihu
!mkdir Dataset/nahjalbalagha
!mkdir Dataset/train
!mkdir Dataset/test
!mkdir Dataset/validation

In [ ]:
!mv Dataset/*-zhihu.tsv Dataset/zhihu
!mv Dataset/*-nahjalbalagha.tsv Dataset/nahjalbalagha
!mv Dataset/*-training.tsv Dataset/train
!mv Dataset/*-test.tsv Dataset/test
!mv Dataset/*-validation.tsv Dataset/validation

# Background Information

## Human Value Detection 2023 <br/>
## SemEval 2023 Task 4. ValueEval: Identification of Human Values behind Arguments



Given a textual argument and a human value category, classify whether or not the argument draws on that category. This task uses a set of 20 value categories compiled from the social science literature and described in our [ACL paper](https://webis.de/publications.html#kiesel_2022b). Arguments are given as premise text, conclusion text, and binary stance of the premise to the conclusion ("in favor of" or "against").

The 20 value categories are shown here on Schwartz' value continuum below:

[![JEPBxUu.md.png](https://iili.io/JEPBxUu.md.png)](https://freeimage.host/i/JEPBxUu)







## Data


Data is provided as tab-separated values files with one header line. The arguments-validation.tsv files contain one argument per line: its unique argument ID, the conclusion, the premise's stance towards the conclusion, and the premise itself. Example with tab-separated columns are shown below

<pre><span class="column">Argument ID</span>	<span class="column">Conclusion</span>	<span class="column">Stance</span>	<span class="column">Premise</span>
<span class="column">A01010</span>	<span class="column">We should prohibit school prayer</span>	<span class="column">against</span>	<span class="column">it should be allowed if the student wants to pray as long as it is not interfering with his classes</span>
<span class="column">A01011</span>	<span class="column">We should abolish the three-strikes laws</span>	<span class="column">in favor of</span>	<span class="column">three strike laws can cause young people to be put away for life without a chance to straight out their life</span>
<span class="column">A01012</span>	<span class="column">The use of public defenders should be mandatory</span>	<span class="column">in favor of</span>	<span class="column">the use of public defenders should be mandatory because some people don't have money for a lawyer and this would help those that don't</span>
</pre>

The labels-validation.tsv  files also contain one argument per line: its unique argument ID and one column for each of the 20 value categories with a 1 meaning that the argument resorts to the value category and a 0 that not. Example with tab-separated columns are shown below:

<pre><span class="column">Argument ID</span>	<span class="column">Self-direction: thought</span>	<span class="column">Self-direction: action</span>	<span class="column">Stimulation</span>	<span class="column">Hedonism</span>	<span class="column">Achievement</span>	<span class="column">Power: dominance</span>	<span class="column">Power: resources</span>	<span class="column">Face</span>	<span class="column">Security: personal</span>	<span class="column">Security: societal</span>	<span class="column">Tradition</span>	<span class="column">Conformity: rules</span>	<span class="column">Conformity: interpersonal</span>	<span class="column">Humility</span>	<span class="column">Benevolence: caring</span>	<span class="column">Benevolence: dependability</span>	<span class="column">Universalism: concern</span>	<span class="column">Universalism: nature</span>	<span class="column">Universalism: tolerance</span>	<span class="column">Universalism: objectivity</span>
<span class="column">A01010</span>	<span class="column">1</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
<span class="column">A01011</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">1</span>
<span class="column">A01012</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
</pre>

In addition, there are other datasets for evaluating the robustness of our model: validation-zhihu from the recommendation and hotlist section of the Chinese question-answering website Zhihu, test-nahjalbalagha from and based on the Nahj al-Balagha.



## Evaluation

Runs are evaluated on the basis of F1-score, Precision, and Recall: averaged over all value categories and for each category individually.

## Baseline Results

In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!mkdir baseline
!mkdir output

### 1-Baseline

#### Test Dataset

In [ ]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/test --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

python3: can't open file '/content/1-baseline.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "/home/mj06974/IntroToLLM/Assignment 1/evaluator.py", line 152, in <module>
    writeEvaluation(readLabels(args.inputDataset, prefix="labels-"), readLabels(args.inputRun), args.outputDataset)
  File "/home/mj06974/IntroToLLM/Assignment 1/evaluator.py", line 30, in readLabels
    for labelsBaseName in os.listdir(directory):
FileNotFoundError: [Errno 2] No such file or directory: '/content/Dataset/test/'


head: cannot open '/content/output/evaluation.prototext' for reading: No such file or directory


#### Zhihu

In [ ]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/zhihu/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Security: personal', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Conformity: interpersonal', 'Conformity: rules', 'Self-direction: thought', 'Face', 'Achievement', 'Security: societal', 'Humility', 'Power: resources', 'Tradition', 'Universalism: objectivity', 'Universalism: tolerance', 'Stimulation', 'Hedonism', 'Benevolence: caring', 'Self-direction: action', 'Power: dominance'}
Writing run file
Reading /content/Dataset/zhihu/labels-validation-zhihu.tsv
Reading /content/baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.2292179045745204"
}
measure {
 key: "Precision"
 value: "0.12944444444444445"
}
measure {
 key: "Recall"
 value: "1.0"
}


#### Nahjalbalagha

In [ ]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/nahjalbalagha/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Self-direction: action', 'Security: personal', 'Security: societal', 'Tradition', 'Achievement', 'Power: resources', 'Universalism: tolerance', 'Hedonism', 'Universalism: nature', 'Self-direction: thought', 'Conformity: interpersonal', 'Universalism: concern', 'Conformity: rules', 'Universalism: objectivity', 'Humility', 'Power: dominance', 'Stimulation', 'Benevolence: dependability', 'Benevolence: caring', 'Face'}
Writing run file
Reading /content/Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading /content/baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.12845882944826426"
}
measure {
 key: "Precision"
 value: "0.0686379928315412"
}
measure {
 key: "Recall"
 value: "1.0"
}


### Random-Baseline


#### Test

In [ ]:
# DO NOT EDIT
!python3 random-baseline.py --inputDataset Dataset/test --outputDataset baseline
!python3 evaluator.py --inputDataset Dataset/test/ --inputRun baseline1 --outputDataset output
!head -n 12 output/evaluation.prototext

Reading Dataset/test/arguments-test.tsv
Labeling 1576 instances
Detected values: {'Humility', 'Face', 'Tradition', 'Stimulation', 'Security: societal', 'Power: resources', 'Benevolence: dependability', 'Universalism: tolerance', 'Self-direction: action', 'Power: dominance', 'Universalism: concern', 'Conformity: rules', 'Benevolence: caring', 'Universalism: objectivity', 'Hedonism', 'Universalism: nature', 'Achievement', 'Self-direction: thought', 'Security: personal', 'Conformity: interpersonal'}
Writing run file


Reading Dataset/test/labels-test.tsv
Traceback (most recent call last):
  File "/home/mj06974/IntroToLLM/Assignment 1/evaluator.py", line 152, in <module>
    writeEvaluation(readLabels(args.inputDataset, prefix="labels-"), readLabels(args.inputRun), args.outputDataset)
  File "/home/mj06974/IntroToLLM/Assignment 1/evaluator.py", line 69, in readLabels
    raise OSError("No labels found in directory '" + directory + "'")
OSError: No labels found in directory 'baseline1'
measure {
 key: "F1"
 value: "0.1619145366405842"
}
measure {
 key: "Precision"
 value: "0.15076157559181208"
}
measure {
 key: "Recall"
 value: "0.17484945850262804"
}


#### Zhihu


In [ ]:
# DO NOT EDIT
!python3 random-baseline.py --inputDataset Dataset/zhihu/ --outputDataset baseline
!python3 evaluator.py --inputDataset Dataset/zhihu/ --inputRun baseline --outputDataset output
!head -n 12 output/evaluation.prototext

Reading Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Face', 'Self-direction: action', 'Power: resources', 'Achievement', 'Power: dominance', 'Universalism: objectivity', 'Conformity: interpersonal', 'Self-direction: thought', 'Conformity: rules', 'Benevolence: dependability', 'Benevolence: caring', 'Universalism: nature', 'Tradition', 'Universalism: tolerance', 'Humility', 'Security: personal', 'Security: societal', 'Universalism: concern', 'Hedonism', 'Stimulation'}
Writing run file


Reading Dataset/zhihu/labels-validation-zhihu.tsv
Reading baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.18565993486656684"
}
measure {
 key: "Precision"
 value: "0.15400517885128007"
}
measure {
 key: "Recall"
 value: "0.23369425958560086"
}


#### Nahjalbalagha

In [ ]:
# DO NOT EDIT
!python3 random-baseline.py --inputDataset Dataset/nahjalbalagha/ --outputDataset baseline
!python3 evaluator.py --inputDataset Dataset/nahjalbalagha/ --inputRun baseline --outputDataset output
!head -n 12 output/evaluation.prototext



Reading Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Achievement', 'Stimulation', 'Security: personal', 'Power: resources', 'Security: societal', 'Conformity: interpersonal', 'Hedonism', 'Universalism: objectivity', 'Benevolence: caring', 'Conformity: rules', 'Benevolence: dependability', 'Power: dominance', 'Humility', 'Self-direction: thought', 'Tradition', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: concern', 'Face', 'Self-direction: action'}
Writing run file


Reading Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.08931047097840349"
}
measure {
 key: "Precision"
 value: "0.06366324442396012"
}
measure {
 key: "Recall"
 value: "0.14956312265615454"
}


# Tasks

## [20 Points] Task 01 - Load Datasets

In this task, you are required to load the Training, Test, Validation, Nahjalbalagha & Zhihu into seperate dataframes.

In [14]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification,BertModel
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from torch import cuda



In [15]:
# Importing files into dataframes:  naming convention: 
# dataset_arguments = the data
# dataset_result_sem_eval = the answers to the data
# dataset_result_level_1 = don't know what that is right now, maybe some answers for some other task. 

# Nahjalbalagah
nahjalbalagha_arguments = pd.read_csv("Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv",sep="\t")
nahjalbalagha_result_sem_eval= pd.read_csv("Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv",sep="\t")
nahjalbalagha_result_level_1= pd.read_csv("Dataset/nahjalbalagha/level1-labels-test-nahjalbalagha.tsv",sep="\t")
# Zhihu 
zhihu_arguments = pd.read_csv("Dataset/zhihu/arguments-validation-zhihu.tsv",sep="\t")
zhihu_result_sem_eval = pd.read_csv("Dataset/zhihu/labels-validation-zhihu.tsv",sep="\t")
zhihu_result_level_1 = pd.read_csv("Dataset/zhihu/level1-labels-validation-zhihu.tsv",sep="\t")

# Testing 
test_arguments = pd.read_csv("Dataset/test/arguments-test.tsv",sep="\t")
test_result_sem_eval = pd.read_csv("Dataset/test/labels-test.tsv",sep="\t")
test_result_level_1 = pd.read_csv("Dataset/test/level1-labels-test.tsv",sep="\t")

# Training Dataset
training_arguments = pd.read_csv("Dataset/train/arguments-training.tsv",sep="\t")
training_result_sem_eval = pd.read_csv("Dataset/train/labels-training.tsv",sep="\t")
training_result_level_1 = pd.read_csv("Dataset/train/level1-labels-training.tsv",sep="\t")

# Validation 
validation_arguments = pd.read_csv("Dataset/validation/arguments-validation.tsv",sep="\t")
validation_result_sem_eval = pd.read_csv("Dataset/validation/labels-validation.tsv",sep="\t")
validation_result_level_1 = pd.read_csv("Dataset/validation/level1-labels-validation.tsv",sep="\t")


##  Merging Dataframes


In [16]:

label_dict={
    0: "",
    1: "Self-direction: thought",
    2: "Self-direction: action",
    3: "Stimulation",
    4: "Hedonism",
    5: "Achievement",
    6: "Power: dominance",
    7: "Power: resources",
    8: "Face",
    9: "Security: personal",
    10: "Security: societal",
    11: "Tradition",
    12: "Conformity: rules",
    13: "Conformity: interpersonal",
    14: "Humility",
    15: "Benevolence: caring",
    16: "Benevolence: dependability",
    17: "Universalism: concern",
    18: "Universalism: nature",
    19: "Universalism: tolerance",
    20: "Universalism: objectivity"
}
label_dict_inverse = {v: k for k, v in label_dict.items()}

argument_values = {}

# Iterate over each row in the dataframe
for index, row in training_result_sem_eval.iterrows():
    # Get the 'Argument ID'
    argument_id = row['Argument ID']
    # Initialize an empty list to store column names with value 1
    columns_with_1 = []
    # Iterate over each column in the row (starting from index 1 to skip 'Argument ID')
    for col in training_result_sem_eval.columns[1:]:
        # Check if the value in the current column is 1
        if row[col] == 1:
            # If so, append the column name to the list
            columns_with_1.append(label_dict_inverse[col])
    # Add the 'Argument ID' and list of columns with value 1 to the dictionary
    argument_values[argument_id] = columns_with_1
df_argument_values = pd.DataFrame(argument_values.items(), columns=['Argument ID', 'category'])

merged_df = pd.merge(training_arguments, df_argument_values, on='Argument ID', how='left')





## [10 Points] Task 02 - Define Tokenizer & Model


In this task, you are required to define the Tokenizer and LLM model of your choice.

In [28]:

# Write your code here
device = 'cuda' if cuda.is_available() else 'cpu'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',num_labels=20)
model = BertModel.from_pretrained("bert-base-uncased",id2label=label_dict,label2id=label_dict_inverse)
model.to(device)



text_data = merged_df.apply(lambda row: ','.join(map(str, row)), axis=1).tolist()
# print(type(text_data))
data=[tokenizer(text,padding=True) for text in text_data]

# print(data)
# Reference # 1

{'input_ids': [101, 1037, 24096, 8889, 2475, 1010, 2057, 2323, 7221, 2529, 18856, 13369, 1010, 1999, 5684, 1997, 1010, 2057, 2323, 7221, 2529, 18856, 13369, 2004, 2009, 2097, 2069, 3426, 4121, 3314, 2043, 2017, 2031, 1037, 9129, 1997, 1996, 2168, 4286, 2770, 2105, 2035, 3772, 1996, 2168, 1012, 1010, 1031, 2184, 1033, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## [20 Points] Task 03 - Optimizer & Hyperparameters


In this task, you are required to define the hyperparameters & the optimizer for training your model.

In [18]:
# Write your code here
BATCH_SIZE = 32
NUM_PROCS = 32
LR = 0.00005
EPOCHS = 5
MODEL = 'bert-base-uncased'
OUT_DIR = 'finetuned '+ MODEL

## [20 Points] Task 04 -  Training Loop


In this task, you are required to implement the training loop for fine tuning your model. You are also required to plot on the same graph: Loss vs Epochs & Accuracy vs Epochs

In [19]:
# Write your code here

    

## [10 Points]  Task 05 - Model Evaluation: Test Dataset

In this task, you are required your fine tuned model on the Test dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

In [20]:
#Write your code

## [10 Points]  Task 06 - Model Evaluation: Zhihu Dataset

In this task, you are required your fine tuned model on the Zhihu
 dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

## [10 Points]  Task 07 - Model Evaluation: Nahjalbalagha Dataset

In this task, you are required your fine tuned model on the Nahjalbalagha dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

# References

In this section, cite any resources or references that you use for solving this assignment.


title = {{BERT:} Pre-training of Deep Bidirectional Transformers for Language Understanding}
Link : https://huggingface.co/google-bert/bert-base-uncased 

